In [ ]:
import pyodbc
import sys

class Hotel:
     
    def __init__(self):
        try:
            self.conn = pyodbc.connect(
                'driver={SQL Server};'
                'Server=LAPTOP-S0GFKI1C\\SQLEXPRESS;'
                'Database=Hotel;'
                'Trusted_Connection=yes'
            )

            self.cursor = self.conn.cursor()
            self.cursor.execute("SELECT * FROM menu")
            self.records = self.cursor.fetchall()
        except:
          print(sys.exc_info())

    def waiter(self):
        """ For show the menu """
        print("\n" + "-"*40)
        print("{:^40}".format("WELCOME TO HOTEL"))
        print("-"*40)
        print("{:<5}{:<15}{:>10}".format("ID","Dish","Price"))

        for r in self.records:
            print("{:<5}{:<15}{:>10}".format(r[2],r[0],r[1]))

        print("-"*40)


class Customer(Hotel):

    def __init__(self):
        super().__init__()

        self.items=[]
        self.qty=[]
        self.prices=[]
        self.waiter()

    def order(self):
        """For taking the orders """
        try:
            table_no=int(input("Enter table number: "))

            while True:
                dish=input("Enter dish name ")
                quantity=int(input("Enter quantity: "))

                self.cursor.execute(
                    "SELECT dish,price FROM menu WHERE dish=?",
                    dish
                )
                row=self.cursor.fetchone()
                self.cursor.execute(
                        "SELECT menu_id FROM menu WHERE dish=?",
                        dish
                    )
                item_id = int(self.cursor.fetchone()[0])
                if row:

                    name=row[0]
                    price=row[1]
                    total=price*quantity

                    self.items.append(name)
                    self.qty.append(quantity)
                    self.prices.append(total)

                    self.cursor.execute("SELECT ISNULL(MAX(order_id),0)+1 FROM orders")
                    

                    self.cursor.execute("SELECT ISNULL(MAX(order_id),0)+1 FROM orders")
                    

                    self.cursor.execute(
                        "INSERT INTO orders(Menu_id, quantity, creation_date) VALUES (?, ?, GETDATE())",
                        (item_id, quantity)
                    )


                    print("Added Saved to DB")

                else:
                    print("Invalid item id ")

                more=input("Add more items? (yes/no): ")
                if more!="yes":
                    break

            self.conn.commit()
            self.print_bill(table_no)
        except:
          print(sys.exc_info())


    def print_bill(self,table_no):
        """For printing the bills """

        if input("you want hard copy: ")=="yes":
            a=input("enter hotel name")
            s=0
            e= "-------------------------------------------------"
            f="|"
            print(e)
            print("{:^40}".format(a))
            print(e)
            print("table no.:",table_no)
            print(e)
            print("{:<2}".format(f),end="")
            print("{:<5} {:<12} {:>8} {:>8}".format("sr","menu","quant","price"),end="")
            print("{:>4}".format(f))
            for i in range(len(self.items)):
                print("{:<2}".format(f),end=" ")
                print("{:<5} {:<12} {:>8} {:>8}".format(i+1,self.items[i],self.qty[i],self.prices[i]),end="")
                print("{:>2}".format(f))
                s=s+self.prices[i]
            print(e)
            print("{:<20}".format("Total"),end="")
            print("{:>20}".format(str(s)))
        else:
            print("ok visit again.....")


In [6]:
c1=Customer()
c1.order()


----------------------------------------
            WELCOME TO HOTEL            
----------------------------------------
ID   Dish                Price
1    Paneer masala    120.0000
2    Paneer Maharaja  200.0000
3    shev bhaji       150.0000
4    Kaju curry       120.0000
5    chapati           10.0000
6    tandoor roti      20.0000
7    Steam rice        90.0000
8    jeera rice       120.0000
9    dal fry          100.0000
10   dal tadka        120.0000
----------------------------------------
Added Saved to DB
-------------------------------------------------
                Shresha                 
-------------------------------------------------
table no.: 1
-------------------------------------------------
| sr    menu            quant    price   |
|  1     Paneer masala         2 240.0000 |
-------------------------------------------------
Total                           240.0000
